<a href="https://colab.research.google.com/github/AJamal27891/AJamal/blob/master/NLP_AMAZON_REVIEWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 
**The main objective is to detrmined if the review is positive or negative and to do that we need the following :**


---


1.   data preprocessing (tokenaize the setnence , remove stop words, lammatize data)

2.    pretrained model (word2vector )

3.   feed the pretrained model to (RNN model ) 

4.   train the model and report the test results. 


In [0]:
#@title Important Imports 
import random
import re # regular expression
from collections import Counter 
from functools import partial
from pathlib import Path
import gensim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google_drive_downloader import GoogleDriveDownloader as gdd
from IPython.core.display import display, HTML
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF
from sklearn.metrics import classification_report
from tqdm import tqdm, tqdm_notebook # show progress bar
from keras.optimizers import Adam
# PyTorch modules
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

# nltk text processors
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.corpus import stopwords
import spacy
from nltk.tokenize import sent_tokenize
from google.colab import drive
from google.colab import files

Using TensorFlow backend.


In [0]:
#@title Enable GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [0]:
#@title Connect with GDrive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#@title Stop words and tokenization
nltk.download('stopwords')
sw = stopwords.words('english')
nlp = spacy.load('en',disable=['parser','tagger','ner'])
tokenizer = RegexpTokenizer("\w+\'?\w+|\w+")
sw = stopwords.words('english')
sw2 = STOP_WORDS
nlp = spacy.load('en',disable=['parser','tagger','ner'])
stop_words = set(sw).union(sw2)
finalstop = {'these', 'does', 'beyond', 'wasn', 'nowhere', 'former', 'except', 'herself', 'too', 'everyone', 'namely', 'well', 'therefore', 'side',
                  'amongst', 'much', 'others', 'empty', 'having', 'even', 'next', 'per', 'were', 'whereupon', 'the', 'haven', 'make', 'him', 'becomes',
                  'seemed', 'throughout', 'ourselves', 'shan', 'about', 'third', 'she', 'somewhere', 'thereupon', 'name', 'be', 'hers','nine', 'nevertheless',
                  'towards','made', 'off', 'into', 'hereupon','do', 'sometimes', 'therein', 'which','enough', 'o', 'ma', 'while','show', 'call', 'full', 'though',
                  'top', 'this', 'may', 'anyone', "'m", 'between', 'then', 'up', 'everywhere', 'or', 'latter', 'ours', 'once', 'whereafter', 'been', 'same','could',
                  'they', 'whole', 'as', 'eight', 'll', 'who', 'everything', 'three', 'meanwhile', 'might', 'now', 'an','two', 'together', 'from', 'eleven',
                  'under', 'how', 'seeming', 'at', 'ten', 'thereby', 'before', 'amount', 'in', 'along', 'several', 'latterly', 'rather', 'them', 'noone', 'yet',
                  'after', 'quite', 'more', 'such', 'perhaps', 'each','hereby', 'there', 'fifty', 'own', 's', 'forty', 'serious', 'whereby', 'until', 'formerly',
                  'twenty', 'take', 'can', 'if', 'also', 'yourselves', 'twelve', 'whereas', 'ca', 'almost', 'below', 'among', 'always', 'one', 'fifteen', 'me',
                  'should', 'four', "that'll",'back', 'only', 'put', 'across', 'few', 'whenever', 'when', 'mightn', 'around', 'itself', 'above', 'doing', 'will', '‘ll',
                  'by', 'elsewhere', 'further', 'get','go', 'hence', 'sixty', 'my', 'part', 'and', 'during', 'for', 'indeed', 'anyway', 'give', 'must', 'was', 'someone',
                  'wouldn', 'hadn', 'would', 'upon', 'wherever', 'keep', 'either', '‘d', 'against', 'i', 'has', 'most', "'re", 'yourself', 'cannot', 'five', '‘ve','see',
                  'became', 'first', 'whatever', 'afterwards','regarding', 'a','his', 'nothing', 'through', 'your', 'besides', 'within', 'it', "should've", 'beforehand',
                  're', 'needn', 'other', 'neither', 'have', 'm', 'last', 'wherein', 'its', 'herein', 'whither', 'had', 'us', 'whence', 'we', 'otherwise', 'our', 'used',
                  'somehow', 'd', 'but', 'he', 'any', 'become', 'alone', 'theirs', 'on', 'being', 'to', 'himself', 'of', 'yours', 'move', 'over', 'thence', "you're",
                  '‘m','where', 'just', 'using', '’re','out', 'because', "'ll", 'none', 'unless', 've', 'whether', 'seems', 'due', 'than', 'aren', 'whoever', 'seem',
                  'anywhere', 'already', 'both', 'something', 'mine','am', 'ever', "you've", 'really', "you'd", 'those', 'y', 'every', '’m', 'since', 'hereafter', 'however',
                  'very', 'often', 'moreover', 'beside', 'with', 'onto', 'her', 'that', 'behind', 'all','bottom', 'thru', 'whose', 'sometime', 'please', 'thus', 'myself',
                  'via', 'why', 'some', 'what', '’s', 'you', 'still', 'various', '‘s', 'mostly', 'front', 'six', 'themselves', 'anyhow', 'thereafter', 'hundred', "'ve", "'s",
                  'becoming', 'their', 'here', 'less', 'so', "you'll", '’ve', 'although', 'say', 'whom', 'mustn', 'toward'}

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
#@title Data Pipeline
def load_data(path,limit=False):
  print('opening the file')
  txt_file = open(path,'r')
  print('extracting features')
  features = list(map(lambda x:x[10:],txt_file.readlines()))
  txt_file.close()
  print('close the file')
  features = pd.DataFrame(features)
  features = features[0].tolist()
  print('opening the file')
  txt_file = open(path,'r')
  print('extracting labels')
  targets = list(map(lambda x:x[0:10],txt_file.readlines()))
  txt_file.close()
  print('closing file')
  targets = pd.DataFrame(targets)
  targets = pd.DataFrame(targets)
  targets = targets[0].tolist()
  print('enlabling lables')
  targets= [ 0 if targets == '__label__2' else 1 for targets in targets ]
  if limit != False:
    print('limiting the data with %s lines ' % limit)
    features = features[:limit]
    targets = targets[:limit]

  print('data load done!')


  return features, targets

def tokenization(reviews):
  return tokenizer.tokenize(reviews)

def remove_stopwords(review):
    return [token for token in review if token not in finalstop]

def limmatization(reviews):
  lemma_resutl=[]
  for words in reviews:
    doc = nlp(words)
    for token in doc:
      lemma_resutl.append(token.lemma_)
  return lemma_resutl

def pipeline(reviews):
  reviews = tokenization(reviews)
  reviews = remove_stopwords(reviews)      
  return reviews

In [0]:
def pipeline_n(data,limma=False):
  print('tokenize the dataset')
  data = list(map(lambda x:tokenizer.tokenize(x.lower()),data))
  print('remove stop words')
  data = list(map(lambda x: remove_stopwords(x),data))
  if limma is True:
    print('limmatiz process ')
    data = list(map(lambda x: limmatization(x),data))
  print('pipline done!')
  return data

In [0]:
X, y = load_data('gdrive/My Drive/Colab Notebooks/Amazon/train.ft.txt',limit=300000)

X = pipeline_n(X)

from gensim.models import Word2Vec
model = Word2Vec(X,size=100,min_count=3, workers=2)

word_vec= model.wv
del model 
len(word_vec.vocab)

def word2idx(embedding_model,review):
    index_review = []
    for word in review:
        try:
            index_review.append(embedding_model.vocab[word].index)
        except: 
             pass
    return index_review


X  = list(map(lambda x: word2idx(word_vec,x),X))
padding_value = len(word_vec.index2word)

embedding_weights = word_vec.vectors



opening the file
extracting features
close the file
opening the file
extracting labels
closing file
enlabling lables
limiting the data with 300000 lines 
data load done!
tokenize the dataset
remove stop words
pipline done!


In [0]:
#@title SVM classification is one of the most used classic models Classifiers for stentence and NLP using Word embedding the problem with this model it takes long time to train espically with this large dataset in order to make this code excutable we used less than 10% of the dataset  as it will take forever to train on the 3600000 lines 

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

clf = SVC(degree=2)

print(np.mean(cross_val_score(clf, pd.DataFrame(X).fillna(0),y, cv=5)))



/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

0.5142666669833333


[[55842,
  461,
  8835,
  70,
  283,
  199,
  5520,
  64035,
  245,
  1449,
  36,
  427,
  17099,
  88,
  30,
  376,
  88,
  33986,
  1632,
  637,
  376,
  14,
  30,
  6714,
  120,
  4023,
  50123,
  262,
  15211,
  783,
  8467,
  2761,
  3953,
  16855,
  4741,
  2139,
  162,
  9057],
 [14,
  805,
  126,
  28,
  40,
  67,
  113,
  583,
  0,
  14,
  88,
  805,
  1416,
  242,
  307,
  127,
  2684,
  123,
  0,
  55843,
  1651,
  788,
  30,
  0,
  1763,
  28,
  358,
  44,
  1073,
  228,
  5433,
  5458,
  84,
  3857,
  0,
  112,
  13973,
  3,
  69,
  19,
  18,
  38,
  0,
  101,
  60,
  2221],
 [166,
  805,
  0,
  130,
  30,
  10,
  676,
  59,
  1971,
  4985,
  9195,
  4279,
  705,
  376,
  88,
  218,
  5448,
  2120,
  357,
  5319,
  144,
  470,
  2266,
  994,
  1784,
  44,
  1304,
  1258,
  308,
  4,
  33986,
  1632,
  19031,
  13693,
  10,
  50124,
  33986,
  4999,
  3,
  268,
  968,
  805,
  0,
  166,
  30,
  158,
  14,
  28924,
  24,
  404,
  149,
  64036,
  805,
  56,
  114,
  29,
  376

In [0]:
#@title Naive Bayes approach is another classic Model for sentence classification accuracy on the test set after training is 50% 

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_sc = scaler.fit_transform(X)
clf3 = GaussianNB()
print('NaiveBayes' ,np.mean(cross_val_score(clf3,X_sc ,y, cv=5)))


ValueError: ignored

In [0]:
#@title  Random Forest Classifier using Word embedding the validation accuracy is 52% which as bad as predicted Random Forest is not a good classifier for Sentences the test set is 55% accuracy after trainig.
X, y = load_data('gdrive/My Drive/Colab Notebooks/Amazon/train.ft.txt',limit=200000)

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec

X = pipeline_n(X)

model = Word2Vec(X,size=100,min_count=3, workers=2)

word_vec= model.wv
del model 
len(word_vec.vocab)

def word2idx(embedding_model,review):
    index_review = []
    for word in review:
        try:
            index_review.append(embedding_model.vocab[word].index)
        except: 
             pass
    return index_review


X  = list(map(lambda x: word2idx(word_vec,x),X))
scaler = StandardScaler()
X_sc = scaler.fit_transform(pd.DataFrame(X).fillna(0))
clf2 = RandomForestClassifier(n_estimators=200)
print('RandomForest',np.mean(cross_val_score(clf2, X_sc,y, cv=5)))



opening the file
extracting features
close the file
opening the file
extracting labels
closing file
enlabling lables
limiting the data with 2000000 lines 
data load done!
tokenize the dataset
remove stop words
pipline done!


KeyboardInterrupt: ignored

In [0]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip -q glove.840B.300d.zip

--2019-12-23 13:04:14--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2019-12-23 13:04:14--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2019-12-23 13:04:14--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

In [0]:
#@title In this section we will use GloVe(globel Vector) in our deep learning instead of our Word2vec for more gerelized results Preparing the data for the CNN and RNN models Using Glove with 300D which transfer the words into global vectors of 300 so it will present a menaing to our model instead of plain text.

from __future__ import print_function, division 
from builtins import range 

import os 
import sys 
from  keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from sklearn.metrics import roc_auc_score


MAX_SEQUENCE_LENGTH = 100 
MAX_VOCAB_SIZE = 20000
EMBEDDING_DIM = 300 
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128 
EPOCHS = 10 

print('Loading word Vectors ......')
word2vec={}
with open('gdrive/My Drive/Colab Notebooks/glove.6B.300d.txt',encoding = 'utf-8')as f:
        for line in f:
                values = line.split()
                word = values[0]
                vec = np.asarray(values[1:],dtype=float)
                word2vec[word] = vec
        print('Found %s word vectors .' % len(word2vec))




Loading word Vectors ......


FileNotFoundError: ignored

In [0]:
#@title prepare text samples and thier lables sequence the data and apply the pipe line one it.
X, y = load_data('gdrive/My Drive/Colab Notebooks/Amazon/train.ft.txt',limit = 1000000)
sequence = X
targets = y

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(sequence)
sequence = tokenizer.texts_to_sequences(sequence)
word2idx = tokenizer.word_index

opening the file
extracting features
close the file
opening the file
extracting labels
closing file
enlabling lables
limiting the data with 200000 lines 
data load done!


In [0]:
#@title tokenize the data and create word embedding using GLove.


print('found %s unique tokenz.' % len(word2idx))



data = pad_sequences(sequence, maxlen =MAX_SEQUENCE_LENGTH )

print('shape of the data tensor', data.shape)

print('filling pretrained embedding')
num_words = min(MAX_VOCAB_SIZE,len(word2idx)+1)
embedding_matrix = np.zeros((num_words,EMBEDDING_DIM))

for word, i in word2idx.items():
    if i < MAX_VOCAB_SIZE:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i]= embedding_vector

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights = [embedding_matrix],
                            input_length = MAX_SEQUENCE_LENGTH,
                            trainable= False)


found 184503 unique tokenz.
shape of the data tensor (200000, 100)
filling pretrained embedding



In [0]:
#@ The model is consists of 3 convolusion layers , 3 relu layers, and 2 maxpooling and one global max pooling with loss binary_crossentropy and adam as optimizer
print('building Model')

input_ = Input(shape= (MAX_SEQUENCE_LENGTH,))
x = embedding_layer(input_)
x = Conv1D(128,3,activation  = 'relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128,3,activation  = 'relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128,3,activation  = 'relu')(x)
x = GlobalMaxPooling1D()(x)
output = Dense(2,activation  = 'sigmoid')(x)

model = Model(input_,output)
model.compile(optimizer=Adam(lr=0.01),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
print('training model')

r = model.fit(data,targets,
              batch_size = BATCH_SIZE,
              validation_split= VALIDATION_SPLIT,
              epochs = EPOCHS)

import matplotlib.pyplot as plt
plt.plot(r.history['loss'],label='loss')
plt.plot(r.history['val_loss'],label='val_loss')
plt.legend()
plt.show()

plt.plot(r.history['accuracy'],label='accuracy')
plt.plot(r.history['val_accuracy'],label='val_accuracy')
plt.legend()
plt.show()



p = model.predict(data)

auc = roc_auc_score(targets,p)

print(np.mean(aucs))

In [0]:
sequence[0]

In [0]:
#@title Create Long short memorty model contains of 15 units and one globalmaxpooling and last dense fully connected layer.
print('Building LSTM.......')
input_ = Input(shape= (MAX_SEQUENCE_LENGTH,))
x = embedding_layer(input_)
x = Bidirectional(LSTM(15,return_sequences=True))(x)

x = GlobalMaxPooling1D()(x)

output = Dense(1,activation  = 'sigmoid')(x)

model = Model(input_,output)
model.compile(optimizer=Adam(lr=0.01),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
print('training model')

r = model.fit(data,targets,
              batch_size = BATCH_SIZE,
              validation_split= VALIDATION_SPLIT,
              epochs = EPOCHS)

import matplotlib.pyplot as plt
plt.plot(r.history['loss'],label='loss')
plt.plot(r.history['val_loss'],label='val_loss')
plt.legend()
plt.show()

plt.plot(r.history['accuracy'],label='accuracy')
plt.plot(r.history['val_accuracy'],label='val_accuracy')
plt.legend()
plt.show()



p = model.predict(data)

auc = roc_auc_score(targets,p)

print(np.mean(aucs))


Building LSTM.......


NameError: ignored